In [9]:
import numpy as np
import torch

# numpy中的广播

In [3]:
# 一般情况下 矩阵的加减和点积需要在相同维度的两个矩阵上进行

a = np.array([1.0, 2.0, 3.0])
b = np.array([4.0, 5.0, 6.0])
a*b

array([ 4., 10., 18.])

In [4]:
# 下方是一个简单的广播样例

c = np.array([1.0, 2.0, 3.0])
d = 4.0
c*d

array([ 4.,  8., 12.])

In [7]:
# (4,1)与(3)进行广播 相当于(4,1)与(1,3)广播 结果为(4,3)

a = np.array([0.0, 10.0, 20.0, 30.0])
b = np.array([1.0, 2.0, 3.0])

c = a[:, np.newaxis] + b
print(c.shape)
c

(4, 3)


array([[ 1.,  2.,  3.],
       [11., 12., 13.],
       [21., 22., 23.],
       [31., 32., 33.]])

# pytorch中的广播

In [17]:
a = torch.tensor([0.0, 10.0, 20.0, 30.0])
a = a.unsqueeze(dim=1)
b = torch.tensor([1.0, 2.0, 3.0])

In [18]:
print(a.shape)
print(b.shape)

torch.Size([4, 1])
torch.Size([3])


In [19]:
c = a*b
c.shape

torch.Size([4, 3])

## masked_fill使用广播

In [26]:
# 假设一个tensor x进行self-attention计算 已经计算得到key query
query = torch.randn(size=(2, 3, 8))
key = torch.randn(size=(2, 3, 8))
# query与key维度为[b, s, d] batch=2 seq_len=3 d_model=8

In [27]:
# 计算attention score
attention_score = torch.matmul(query, key.transpose(-1, -2))
attention_score.shape
# attention_score为2,3,3 即对于seq上的每个位置 都对其他三个位置有attention

torch.Size([2, 3, 3])

In [28]:
# 而对于tensor有pad mask
mask = torch.tensor([[1.0, 0.0, 0.0],
                     [1.0, 1.0, 0.0]])
mask.shape
# 其中0.0表示需要mask的位置 1.0表示不需要mask的位置
# mask的含义是batch1的后面两个token都需要mask  batch2的第三个token需要mask

torch.Size([2, 3])

In [29]:
mask_bool = mask == 0.
mask_bool

tensor([[False,  True,  True],
        [False, False,  True]])

In [30]:
# 现在attention score与mask_bool不满足广播要求
attention_score.masked_fill(mask_bool, -1e9)

# 因为attention_score为(2,3,3) 而mask_bool为(2,3)
# 从右向左看3与2不相等

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

In [31]:
# 此时从mask本来含义入手，mask的第一个值需要对第一个batch中的每一个seq位置的第二个和第三位置遮盖
# 对于seq=0的三个注意力值而言，需要对第二个和第三个mask 对于seq=1和2也完全相同
# 那么将mask_bool在seq维度上扩展，让其广播至每个seq_len上即可
mask_bool_ = mask_bool.unsqueeze(dim=1)
mask_bool_.shape

torch.Size([2, 1, 3])

In [33]:
attention_score_ = attention_score.masked_fill(mask_bool_, -1e9)
attention_score_
# 满足条件

tensor([[[-1.7514e+00, -1.0000e+09, -1.0000e+09],
         [ 1.4653e+00, -1.0000e+09, -1.0000e+09],
         [ 4.5117e+00, -1.0000e+09, -1.0000e+09]],

        [[-1.5321e+00, -1.3974e+00, -1.0000e+09],
         [ 5.8771e+00, -2.3576e+00, -1.0000e+09],
         [-5.3886e-01,  3.5407e+00, -1.0000e+09]]])